In [1]:
# !pip install ccxt
# !pip install slacker
import ccxt
import pprint
from datetime import datetime
from slacker import Slacker
import time

In [2]:
# 파일로부터 apiKey, Secret 읽기 
with open("apiKey_ham.txt") as f:
    lines = f.readlines()
    apiKey = lines[0].strip() 
    secret = lines[1].strip() 
    slackToken = lines[2].strip()

In [3]:
slack = Slacker(slackToken)
binance = ccxt.binance()

In [4]:
# 슬랙 메세지 보내기
# slack.chat.post_message('#autostock','hi bum!')

In [18]:
# 선물 잔고 조회
# free : 보유중, used : 거래진행중, 
binance = ccxt.binance({
    'apiKey': apiKey,
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})
balance = binance.fetch_balance()
print(balance['USDT'])
print(type(balance["USDT"]["free"]))

{'free': 621.29971885, 'used': 0.0, 'total': 621.29971885}
<class 'float'>


In [17]:
# 현재가 조회
ticker = binance.fetch_ticker('ETH/USDT')
print(ticker['open'], ticker['high'], ticker['low'], ticker['close'])
print(type(ticker["close"]))

4703.79 4765.0 4452.53 4522.01
<class 'float'>


---

In [5]:
def showDateTime(dateTime):
    return datetime.fromtimestamp(dateTime/1000).strftime('%Y-%m-%d %H:%M:%S')

In [13]:
# ohlcv
eth5m = binance.fetch_ohlcv(
    symbol="ETH/USDT", 
    timeframe='5m', 
    since=None, 
    limit=20)
for e in eth5m:
    print(e[0])
    print(showDateTime(e[0]), e[1:])

1635043200000
2021-10-24 11:40:00 [4141.78, 4146.0, 4130.0, 4130.28, 2225.667]
1635043500000
2021-10-24 11:45:00 [4130.28, 4138.88, 4128.28, 4138.41, 2019.61]
1635043800000
2021-10-24 11:50:00 [4138.4, 4141.0, 4136.0, 4139.51, 1470.445]
1635044100000
2021-10-24 11:55:00 [4139.51, 4147.32, 4136.0, 4143.37, 2984.258]
1635044400000
2021-10-24 12:00:00 [4143.28, 4143.28, 4134.8, 4136.51, 2202.705]
1635044700000
2021-10-24 12:05:00 [4136.51, 4146.92, 4135.7, 4146.82, 3010.063]
1635045000000
2021-10-24 12:10:00 [4146.82, 4149.94, 4145.08, 4147.82, 2360.447]
1635045300000
2021-10-24 12:15:00 [4147.82, 4147.82, 4141.1, 4142.84, 1391.503]
1635045600000
2021-10-24 12:20:00 [4142.83, 4150.87, 4141.55, 4145.76, 1906.978]
1635045900000
2021-10-24 12:25:00 [4145.75, 4146.52, 4138.0, 4138.48, 3048.473]
1635046200000
2021-10-24 12:30:00 [4138.49, 4146.0, 4135.2, 4144.96, 2655.848]
1635046500000
2021-10-24 12:35:00 [4144.96, 4148.65, 4141.76, 4146.49, 2346.572]
1635046800000
2021-10-24 12:40:00 [4146.8

In [112]:
def checkIfGoodToBuy():
    eth5m = binance.fetch_ohlcv(
        symbol="ETH/USDT",
        timeframe='5m',
        since=None,
        limit=10)
    candleL = []
    for e in eth5m:
        candle = e[4] - e[1]
        if candle > 3:
            candleL.append("white")
        elif candle < -3:
            candleL.append("black")
        else:
            candleL.append("")
    print(candleL)
    if len(candleL) < 3:
        return False
    goodStack = 0
    for i in range(len(candleL) - 1, -1, -1):
        if candleL[i] == "white":
            return False
        elif candleL[i] == "black":
            goodStack += 1
        elif candleL[i] == "" and goodStack >= 3:
            return True
        if goodStack >= 4:
            return True
    return False
checkIfGoodToBuy()

['', 'black', 'black', '', 'black', 'white', '', '', 'black', '']


False

---

In [ ]:
json

In [33]:
# 매수 매도
order = binance.create_market_buy_order(
    symbol="ETH/USDT",
    amount=0.002, 
)
pprint.pprint(order)
 
# order = binance.create_market_sell_order(
#     symbol="ETH/USDT",
#     amount=0.002, 
# )
# pprint.pprint(order)


{'amount': 0.002,
 'average': 3849.86,
 'clientOrderId': 'x-xcKtGhcu27a24d72355b4a0c810ef1',
 'cost': 7.69972,
 'datetime': None,
 'fee': None,
 'fees': [],
 'filled': 0.002,
 'id': '8389765509100861083',
 'info': {'avgPrice': '3849.86000',
          'clientOrderId': 'x-xcKtGhcu27a24d72355b4a0c810ef1',
          'closePosition': False,
          'cumQty': '0.002',
          'cumQuote': '7.69972',
          'executedQty': '0.002',
          'orderId': '8389765509100861083',
          'origQty': '0.002',
          'origType': 'MARKET',
          'positionSide': 'BOTH',
          'price': '0',
          'priceProtect': False,
          'reduceOnly': False,
          'side': 'BUY',
          'status': 'FILLED',
          'stopPrice': '0',
          'symbol': 'ETHUSDT',
          'timeInForce': 'GTC',
          'type': 'MARKET',
          'updateTime': '1634482056016',
          'workingType': 'CONTRACT_PRICE'},
 'lastTradeTimestamp': None,
 'postOnly': False,
 'price': 3849.86,
 'remaining

In [51]:
def getTodayDate():
    now = datetime.now()
    nowDate = now.strftime('%Y%m%d')
    return nowDate

'20211020'

In [66]:

price = 3849.86
updateTime = '1634482056016'

fileName = getTodayDate()+".txt"
try:
    with open(fileName) as f:
        lines = f.readlines()
except:
    f = open(fileName, "w")
    f.close()
    lines = []
lines.append(updateTime + "\n")
with open(fileName, "w") as f:
    for line in lines:
        f.write(line)


In [10]:
# 현재 포지션 확인
balance = binance.fetch_balance()
position = [x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]
print(position)

{'symbol': 'ETHUSDT', 'initialMargin': '0', 'maintMargin': '0', 'unrealizedProfit': '0.00000000', 'positionInitialMargin': '0', 'openOrderInitialMargin': '0', 'leverage': '10', 'isolated': True, 'entryPrice': '0.0', 'maxNotional': '4000000', 'positionSide': 'BOTH', 'positionAmt': '0.000', 'notional': '0', 'isolatedWallet': '0', 'updateTime': '1635437233699'}


In [6]:
balance = binance.fetch_balance()
entryPrice = float([x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]["entryPrice"])
entryPrice

4167.03

In [104]:

balance = binance.fetch_balance()
float([x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]["positionAmt"])

0.0

In [83]:
binance.trades

{}

In [79]:
datetime.now()

datetime.datetime(2021, 10, 20, 2, 22, 47, 919297)

In [60]:
# 현재 포지션 2퍼 위에 매도 예약
# targetPrice = round(float(position["entryPrice"]) * 1.0022, 2)
# print(targetPrice)
# binance.create_limit_sell_order(
#     symbol = 'ETH/USDT',
#     amount = 0.002,
#     price = targetPrice
# )

3858.33


{'info': {'orderId': '8389765509101346200',
  'symbol': 'ETHUSDT',
  'status': 'NEW',
  'clientOrderId': 'x-xcKtGhcu41680a6da6274810a76ddb',
  'price': '3858.33',
  'avgPrice': '0.00000',
  'origQty': '0.002',
  'executedQty': '0',
  'cumQty': '0',
  'cumQuote': '0',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'reduceOnly': False,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '0',
  'workingType': 'CONTRACT_PRICE',
  'priceProtect': False,
  'origType': 'LIMIT',
  'updateTime': '1634483299501'},
 'id': '8389765509101346200',
 'clientOrderId': 'x-xcKtGhcu41680a6da6274810a76ddb',
 'timestamp': 1634483299501,
 'datetime': '2021-10-17T15:08:19.501Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'sell',
 'price': 3858.33,
 'stopPrice': None,
 'amount': 0.002,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 0.002,
 'status': 'open',
 'fee': None,
 'trades': [],
 

In [6]:
# 대기 주문 조회
open_orders = binance.fetch_open_orders(
    symbol="ETH/USDT"
)
pprint.pprint(open_orders)

[]


In [59]:
# 모든 대기 주문 취소
binance.cancel_all_orders(symbol = "ETH/USDT")

{'code': '200', 'msg': 'The operation of cancel all open order is done.'}

In [9]:
eth15m = binance.fetch_ohlcv(
    symbol="ETH/USDT",
    timeframe='15m',
    since=None,
    limit=2)
eth15m

[[1634787000000, 4153.31, 4177.86, 4145.61, 4176.14, 6891.7411],
 [1634787900000, 4176.15, 4192.0, 4175.53, 4183.41, 2199.1437]]

In [98]:
# 물타기
fileName = getTodayDate() + ".txt"
with open(fileName) as f:
    lines = f.readlines()
eth15m = binance.fetch_ohlcv(
    symbol="ETH/USDT",
    timeframe='15m',
    since=None,
    limit=2)
candle = eth15m[0]
if int(lines[-1].split()[0]) > candle[0]:
    print("이전 봉에 구매 이력 있어서 물타기 스킵")
else:
    if candle[4] - candle[1] < -3:
        print("물타기 하기~")
        water()
    else:
        print("이번 봉은 음봉이 아니라 물타기 스킵")
eth15m[0]
print(time.time())
eth15m

이전 봉에 구매 이력 있어서 스킵
1634664650.381064


[[1634663700000, 3821.2, 3823.57, 3812.44, 3813.35, 10643.144],
 [1634664600000, 3813.35, 3814.0, 3808.63, 3809.93, 1806.216]]

---


In [3]:
# record 관리
recordFilePath = "record.txt"
def writeRecord(updateTime):
    with open(recordFilePath, "w") as f:
        f.write(updateTime)

In [13]:
with open(recordFilePath) as f:
    lines = f.readlines()

In [14]:
lines

['1635007505293']

In [12]:
writeRecord("1635007505293")

---

물 탔을때 return 값

{'info': {'orderId': '8389765509611441354', 'symbol': 'ETHUSDT', 'status': 'NEW', 'clientOrderId': 'x-xcKtGhcu205e4e770f4a9bf663ccbd', 'price': '4217.67', 'avgPrice': '0.00000', 'origQty': '0.020', 'executedQty': '0', 'cumQty': '0', 'cumQuote': '0', 'timeInForce': 'GTC', 'type': 'LIMIT', 'reduceOnly': False, 'closePosition': False, 'side': 'SELL', 'positionSide': 'BOTH', 'stopPrice': '0', 'workingType': 'CONTRACT_PRICE', 'priceProtect': False, 'origType': 'LIMIT', 'updateTime': '1635242106729'}, 'id': '8389765509611441354', 'clientOrderId': 'x-xcKtGhcu205e4e770f4a9bf663ccbd', 'timestamp': 1635242106729, 'datetime': '2021-10-26T09:55:06.729Z', 'lastTradeTimestamp': None, 'symbol': 'ETH/USDT', 'type': 'limit', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'sell', 'price': 4217.67, 'stopPrice': None, 'amount': 0.02, 'cost': 0.0, 'average': None, 'filled': 0.0, 'remaining': 0.02, 'status': 'open', 'fee': None, 'trades': [], 'fees': []}

In [20]:

ticker = get_binance_futures(fetch_only=True)

NameError: name 'get_binance_futures' is not defined

In [ ]:
# TP

# order = binance.create_market_buy_order(
#     symbol="ETH/USDT",
#     amount=minAmt,
# )
params = {'stopPrice' : round(4220, 2)} 
print(params)
order = binance.createOrder(symbol='ETH/USDT', type='TAKE_PROFIT_MARKET', side='BUY', amount = 0.002, params=params)